In [1]:
import json
import os
from typing import List, Tuple

In [2]:
hypothesis = 'pts_transformer_encoder_mlps'

In [3]:
base_configuration = {
    'hypothesis': hypothesis,
    'experiment': None,  # will change below
    'data': {
        'path': '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto/folds_training_dicts.pkl',
        'use_pesto': True,
        'ablation_string': '111111111',
        'max_folds': None
    },
    'model': {
        'name': 'pts_transformer_encoder_mlp',
        'kwargs': {
            'hidden_sizes_mlp_a': None,  # will change below
            'mlp_a_dropout_rate': None,  # will change below
            'hidden_sizes_mlp_c': None,  # will change below
            'mlp_c_dropout_rate': None,  # will change below
            'activation': 'relu',
            'input_shape': (10, 9),
            'max_number_of_patches': 10
        }
    },
    'compile': {
        'optimizer': {
            'name': 'adamw',
            'kwargs': {'learning_rate': 1e-3}
        },
        'loss': {
            'name': 'binary_cross_entropy',
            'kwargs': {}
        }
    },
    'fit': {
        'epochs': 200,
        'batch_size': 256,
        'verbose': 1,
        'n_early_stopping_epochs': 12
    }
}

In [4]:
def mlps_option_to_str(mlps_option: List[Tuple[int, int]]) -> str:
    res = []
    for mlp_option in mlps_option:
        res.append(f'{str(mlp_option[0])}-{str(mlp_option[1])}')
    return '_'.join(res)

In [5]:
def dropout_to_str(dropout_rate: float) -> str:
    return str(dropout_rate).replace('.', '_')

In [6]:
dir_path = f'../data/{hypothesis}'
os.makedirs(dir_path, exist_ok=True)

for mlp_a_dropout_rate in [0.0, 0.1, 0.2]:
    for mlp_c_dropout_rate in [0.0, 0.1, 0.2]:
        for hidden_sizes_mlp_a in [[(1024, 1024), (1024, 1024)], [(512, 512), (512, 512)]]:
            for hidden_sizes_mlp_c in [[(1024, 1024), (1024, 1024)], [(512, 512), (512, 512)]]:
                experiment_name = f'mlp_a_dropout_{dropout_to_str(mlp_a_dropout_rate)}_mlp_c_dropout_{dropout_to_str(mlp_c_dropout_rate)}_hidden_sizes_mlp_a_{mlps_option_to_str(hidden_sizes_mlp_a)}_hidden_sizes_mlp_c_{mlps_option_to_str(hidden_sizes_mlp_c)}'
                configuration = base_configuration.copy()
                configuration['experiment'] = experiment_name
                configuration['model']['kwargs']['mlp_a_dropout_rate'] = mlp_a_dropout_rate
                configuration['model']['kwargs']['mlp_c_dropout_rate'] = mlp_c_dropout_rate
                configuration['model']['kwargs']['hidden_sizes_mlp_a'] = hidden_sizes_mlp_a
                configuration['model']['kwargs']['hidden_sizes_mlp_c'] = hidden_sizes_mlp_c

                with open(f'{dir_path}/{experiment_name}.json', 'w') as f:
                    json.dump(configuration, f, indent=4)